# House Prices 2
Competição do Kaggle sobre a previsão de preço das casa na cidade de Ames, Iowa (Estados Unidos).
Essa é uma segunda resolução deste problema presente aqui no meu portfólio. A intenção aqui é utilizar pipelines e técnicas de feature selection para a otimização dos modelos.

# Importando bibliotecas

In [1]:
# Para tratar os dados
import pandas as pd
import numpy as np

# Pré-processamento
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer # pipeline com colunas de tipos diferentes
from sklearn.impute import SimpleImputer # missing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder # escala das features / tratar categóricas numéricas
from category_encoders import TargetEncoder, OneHotEncoder # tratamento de categóricas
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_regression # selecao de features

# Modelagem
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor

# Métricas de avaliação
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# Importando o dataset de treino
houses_train = pd.read_csv('train.csv')
houses_test = pd.read_csv('test.csv')

In [3]:
houses_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
houses_train.shape

(1460, 81)

In [5]:
houses_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# Explorando os dados

In [6]:
# Vamos verificar a quantidade de valores vazios
houses_train.isnull().sum().sort_values(ascending=False).head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
Id                 0
dtype: int64

In [7]:
# Em porcentagem
(houses_train.isnull().sum()/houses_train.shape[0]).sort_values(ascending=False).head(20)

PoolQC          0.995205
MiscFeature     0.963014
Alley           0.937671
Fence           0.807534
MasVnrType      0.597260
FireplaceQu     0.472603
LotFrontage     0.177397
GarageYrBlt     0.055479
GarageCond      0.055479
GarageType      0.055479
GarageFinish    0.055479
GarageQual      0.055479
BsmtFinType2    0.026027
BsmtExposure    0.026027
BsmtQual        0.025342
BsmtCond        0.025342
BsmtFinType1    0.025342
MasVnrArea      0.005479
Electrical      0.000685
Id              0.000000
dtype: float64

In [8]:
# Podemos eliminar as colunas com mais de 20% de valores vazios
eliminar = houses_train.columns[(houses_train.isnull().sum() / houses_train.shape[0]) > 0.2]
eliminar

Index(['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

In [9]:
# Eliminando essas colunas tanto do dataset de treino quanto do dataset de teste
houses_train.drop(eliminar, axis=1, inplace=True)
houses_test.drop(eliminar, axis=1, inplace=True)

In [10]:
(houses_train.isnull().sum()/houses_train.shape[0]).sort_values(ascending=False).head(20)

LotFrontage     0.177397
GarageYrBlt     0.055479
GarageCond      0.055479
GarageType      0.055479
GarageFinish    0.055479
GarageQual      0.055479
BsmtFinType2    0.026027
BsmtExposure    0.026027
BsmtFinType1    0.025342
BsmtCond        0.025342
BsmtQual        0.025342
MasVnrArea      0.005479
Electrical      0.000685
WoodDeckSF      0.000000
PavedDrive      0.000000
LowQualFinSF    0.000000
GrLivArea       0.000000
BsmtFullBath    0.000000
BsmtHalfBath    0.000000
FullBath        0.000000
dtype: float64

In [11]:
houses_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [12]:
# Vamos eliminar também o Id, já que essa coluna é irrelevante
houses_train.drop('Id', axis=1, inplace=True)

In [13]:
# Verificando se há duplicatas nos dados de treino
houses_train.duplicated().sum()

0

In [14]:
# Verificando se há duplicatas nos dados de teste
houses_test.duplicated().sum()

0

Até o momento, removemos algumas colunas que possuíam uma porcentagem alta de valores nulos e também confirmamos que não há duplicatas em nosso conjunto de dados.

# Verificando outliers

In [15]:
# Verificando quais são as colunas numéricas. Vamos aproveitar pra criar a lista de categóricas também.

numerical_columns = houses_train.select_dtypes(include="number").columns.to_list()
categorical_columns = houses_train.select_dtypes(exclude="number").columns.to_list()

In [16]:
print(numerical_columns)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


In [17]:
print(categorical_columns)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']


In [18]:
# Detectando outliers

nomes_colunas = []
qtt_outliers = []

for i in numerical_columns:
    
    contador = 0
    
    q1 = np.quantile(houses_train[i], 0.25) # primeiro quartil
    q3 = np.quantile(houses_train[i], 0.75) # terceiro quartil
    li = q1 - 1.5*(q3-q1) # limite inferior
    ls = q3 + 1.5*(q3-q1) # limite superior
    
    for j in houses_train.index:
        if li <= houses_train[i][j] <= ls:
            pass
        else:
            contador += 1
    
    perc_outliers = (contador / houses_train[i].count())*100 # porcentagem da quantidade de outliers nessa coluna
    
    nomes_colunas.append(i)
    qtt_outliers.append(perc_outliers)

In [19]:
print(nomes_colunas)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']


In [20]:
outliers = pd.DataFrame()
outliers['coluna'] = nomes_colunas
outliers['perc_outliers'] = qtt_outliers
outliers.sort_values(by='perc_outliers', ascending=False)

,coluna,perc_outliers
1,LotFrontage,121.565362
24,GarageYrBlt,105.873822
7,MasVnrArea,100.550964
29,EnclosedPorch,14.246575
9,BsmtFinSF2,11.438356
4,OverallCond,8.561644
31,ScreenPorch,7.945205
0,MSSubClass,7.054795
17,BsmtHalfBath,5.616438
28,OpenPorchSF,5.273973


Há três colunas com uma porcentagem muito alta de outliers. Vamos dar uma olhada em seus valores e tentar descobrir se eles realmente fazem sentido ou devemos excluí-las.

In [21]:
outlier_alto = outliers[outliers['perc_outliers'] > 90]

In [22]:
outlier_alto

,coluna,perc_outliers
1,LotFrontage,121.565362
7,MasVnrArea,100.550964
24,GarageYrBlt,105.873822


In [23]:
for column in outlier_alto['coluna']:
    print('Coluna: ' + column)
    print(f'Média: {houses_train[column].mean()}')
    print(f'Mediana: {houses_train[column].median()}')
    print(f'Min: {houses_train[column].min()}')
    print(f'Max: {houses_train[column].max()}')
    print('---------------------------------\n')

Coluna: LotFrontage
Média: 70.04995836802665
Mediana: 69.0
Min: 21.0
Max: 313.0
---------------------------------

Coluna: MasVnrArea
Média: 103.68526170798899
Mediana: 0.0
Min: 0.0
Max: 1600.0
---------------------------------

Coluna: GarageYrBlt
Média: 1978.5061638868744
Mediana: 1980.0
Min: 1900.0
Max: 2010.0
---------------------------------



A coluna 'LotFrontage' se refere à medida de pés lineares de rua conectada à casa. A coluna 'MasVnrArea' se refere à área folheada de alvenaria em pés quadrados. A coluna 'GarageYrBlt' se refere ao ano em que a garagem foi construída. 

Olhando essas medias e a descrição de cada coluna, é possível notar que faz sentido os intervalos desses valores variarem de tal forma. Vamos então considerar que esses outliers de fato são valores reais do nosso conjunto de dados.

# Tratando variáveis categóricas ordinais

* Verificando a descrição das variáveis no Kaggle, podemos ver quais categóricas são ordinais, então vamos tratá-las agora. Nesse caso, como estamos apenas substituindo os seus valores por um número correspondente sem usar nenhuma informação de outra linha ou coluna, não teremos o risco de data leakage. 
* Obs.: Estamos desconsiderando as colunas que já excluímos anteriormente.

In [24]:
categ_ordinais_na_to_ex = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                  'HeatingQC', 'KitchenQual', 'GarageQual',
                  'GarageCond',]

for column in categ_ordinais_na_to_ex:
    houses_train[column] = houses_train[column].map({'Po':2, 'Fa':3, 'TA':5, 'Gd':7, 'Ex': 9})
    houses_train[column].fillna(0, inplace=True)
    houses_test[column] = houses_test[column].map({'Po':2, 'Fa':3, 'TA':5, 'Gd':7, 'Ex': 9})
    houses_test[column].fillna(0, inplace=True)

houses_train['CentralAir'] = houses_train['CentralAir'].map({'N' :0, 'Y':1})
houses_test['CentralAir'] = houses_test['CentralAir'].map({'N' :0, 'Y':1})

# Criando os modelos

In [25]:
# Vamos selecionar X e y
X = houses_train.drop('SalePrice', axis=1)
y = houses_train.SalePrice

In [26]:
# Separando entre treino e validação. Os dados de teste estão no dataset houses_test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.30, random_state=42)

In [27]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")

X_train shape: (1022, 73)
X_val shape: (438, 73)
y_train shape: (1022,)
y_val shape: (438,)


In [28]:
# Vamos remover a coluna SalePrice da nossa lista de colunas numéricas, já que ela é o nosso target
numerical_columns = [feature for feature in numerical_columns if feature != 'SalePrice']
print(numerical_columns)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [29]:
# Instancia os modelos
lgb_model = lgb.LGBMRegressor()
lr_model = LinearRegression()
rf_model =  RandomForestClassifier()
knn_model = KNeighborsRegressor(n_neighbors=5)

# Preparando os pipelines
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', TargetEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_columns),
    ('cat', categorical_transformer, categorical_columns)
])

# Testando o LGBM

In [30]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('lgbm', lgb_model)
])

# Treina o modelo
pipe.fit(X_train, y_train)

# Cria predições nos dados de validação para avaliar o desempenho
y_pred = pipe.predict(X_val)

# Avalia o modelo
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f'Mean Squared Error (MSE) test:  {mse:.4f}')
print(f'Root Mean Squared Error (RMSE) test:  {rmse:.4f}')
print(f'Mean Absolute Error (MAE) test:  {mae:.4f}')
print(f'R-squared (R2) test:  {r2:.4f}')

Mean Squared Error (MSE) test:  641558588.8579
Root Mean Squared Error (RMSE) test:  25329.0069
Mean Absolute Error (MAE) test:  15701.1724
R-squared (R2) test:  0.9081


# Testando a Regressão Linear

In [31]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('linear_regression', lr_model)
])

# Treina o modelo
pipe.fit(X_train, y_train)

# Cria predições nos dados de validação para avaliar o desempenho
y_pred = pipe.predict(X_val)

# Avalia o modelo
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f'Mean Squared Error (MSE) test:  {mse:.4f}')
print(f'Root Mean Squared Error (RMSE) test:  {rmse:.4f}')
print(f'Mean Absolute Error (MAE) test:  {mae:.4f}')
print(f'R-squared (R2) test:  {r2:.4f}')

Mean Squared Error (MSE) test:  900947211.6966
Root Mean Squared Error (RMSE) test:  30015.7827
Mean Absolute Error (MAE) test:  19744.2250
R-squared (R2) test:  0.8709


# Testando a Random Forest

In [32]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('random_forest', rf_model)
])

# Treina o modelo
pipe.fit(X_train, y_train)

# Cria predições nos dados de validação para avaliar o desempenho
y_pred = pipe.predict(X_val)

# Avalia o modelo
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f'Mean Squared Error (MSE) test:  {mse:.4f}')
print(f'Root Mean Squared Error (RMSE) test:  {rmse:.4f}')
print(f'Mean Absolute Error (MAE) test:  {mae:.4f}')
print(f'R-squared (R2) test:  {r2:.4f}')

Mean Squared Error (MSE) test:  1703387392.8653
Root Mean Squared Error (RMSE) test:  41272.1140
Mean Absolute Error (MAE) test:  25145.9018
R-squared (R2) test:  0.7559


# Testando o KNN

In [33]:
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('KNN', knn_model)
])

# Treina o modelo
pipe.fit(X_train, y_train)

# Cria predições nos dados de validação para avaliar o desempenho
y_pred = pipe.predict(X_val)

# Avalia o modelo
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f'Mean Squared Error (MSE) test:  {mse:.4f}')
print(f'Root Mean Squared Error (RMSE) test:  {rmse:.4f}')
print(f'Mean Absolute Error (MAE) test:  {mae:.4f}')
print(f'R-squared (R2) test:  {r2:.4f}')

Mean Squared Error (MSE) test:  2013320930.5052
Root Mean Squared Error (RMSE) test:  44870.0449
Mean Absolute Error (MAE) test:  29157.8712
R-squared (R2) test:  0.7115


Como pudemos ver, o LGBM foi o modelo que obteve o melhor resultado. Então vamos fazer alguns testes de feature selection com ele e ver se conseguimos melhorar ainda mais as métricas de avaliação.

# Exclusão de features constantes

In [34]:
from feature_engine.selection import DropConstantFeatures

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('drop_constant_features', DropConstantFeatures()),
    ('lgbm', lgb_model)
])

# Treina o modelo
pipe.fit(X_train, y_train)

# Cria predições nos dados de validação para avaliar o desempenho
y_pred = pipe.predict(X_val)

# Avalia o modelo
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f'Mean Squared Error (MSE) test:  {mse:.4f}')
print(f'Root Mean Squared Error (RMSE) test:  {rmse:.4f}')
print(f'Mean Absolute Error (MAE) test:  {mae:.4f}')
print(f'R-squared (R2) test:  {r2:.4f}')

Mean Squared Error (MSE) test:  641558588.8579
Root Mean Squared Error (RMSE) test:  25329.0069
Mean Absolute Error (MAE) test:  15701.1724
R-squared (R2) test:  0.9081


Pelo que pudemos ver, não houve nenhuma mudança nas métricas. Vamos tentar outro método então.

# Exclusão de features correlacionadas

In [35]:
from feature_engine.selection import DropCorrelatedFeatures

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('drop_correlated_features', DropCorrelatedFeatures()),
    ('lgbm', lgb_model)
])

# Treina o modelo
pipe.fit(X_train, y_train)

# Cria predições nos dados de validação para avaliar o desempenho
y_pred = pipe.predict(X_val)

# Avalia o modelo
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f'Mean Squared Error (MSE) test:  {mse:.4f}')
print(f'Root Mean Squared Error (RMSE) test:  {rmse:.4f}')
print(f'Mean Absolute Error (MAE) test:  {mae:.4f}')
print(f'R-squared (R2) test:  {r2:.4f}')

Mean Squared Error (MSE) test:  666639440.7731
Root Mean Squared Error (RMSE) test:  25819.3617
Mean Absolute Error (MAE) test:  15544.4883
R-squared (R2) test:  0.9045


Também não houve nenhuma mudança significativa. Vamos seguir tentando.

# Exclusão de features correlacionadas com SmartCorrelatedSelection

In [36]:
from feature_engine.selection import SmartCorrelatedSelection

scs = SmartCorrelatedSelection(
    method='spearman',
    threshold=0.8,
    missing_values='raise',
    selection_method='variance'
)

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smart_correlated_selection', scs),
    ('lgbm', lgb_model)
])

# Treina o modelo
pipe.fit(X_train, y_train)

# Cria predições nos dados de validação para avaliar o desempenho
y_pred = pipe.predict(X_val)

# Avalia o modelo
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)

print(f'Mean Squared Error (MSE) test:  {mse:.4f}')
print(f'Root Mean Squared Error (RMSE) test:  {rmse:.4f}')
print(f'Mean Absolute Error (MAE) test:  {mae:.4f}')
print(f'R-squared (R2) test:  {r2:.4f}')

Mean Squared Error (MSE) test:  621880271.8434
Root Mean Squared Error (RMSE) test:  24937.5274
Mean Absolute Error (MAE) test:  15174.4472
R-squared (R2) test:  0.9109


Também não houve uma grande mudança, mas podemos ver que o modelo teve uma pequena melhora.
Há outras maneiras de fazer a seleção de features e melhorar nosso modelo. Mas por ora, vamos testar o desempenho nos dados de teste e ver como fica nossa pontuação no Kaggle.

In [37]:
from feature_engine.selection import SmartCorrelatedSelection

scs = SmartCorrelatedSelection(
    method='spearman',
    threshold=0.8,
    missing_values='raise',
    selection_method='variance'
)

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smart_correlated_selection', scs),
    ('lgbm', lgb_model)
])

# Treina o modelo
pipe.fit(X_train, y_train)

# Cria predições nos dados de teste
y_pred = pipe.predict(houses_test)

In [38]:
y_pred.shape

(1459,)

In [39]:
# Adicionando a coluna de previsão na base
houses_test['SalePrice'] = y_pred

In [42]:
# Extraindo somente o Id e o SalePrice
resultado2 = houses_test[['Id', 'SalePrice']]

In [44]:
# Exportando a base
resultado2.to_csv('resultado2.csv', index=False)

# Resultados

O primeiro resultado que havia obtido no Kaggle com o modelo antigo foi de 0.25476, agora com o novo modelo obtivemos 0.13955.